In [ ]:
from itertools import product
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import netCDF4 as nc
import numpy as np
import random
from scipy.integrate import ode
from scipy.interpolate import LinearNDInterpolator

%matplotlib inline

* Staggered grids : Done
* Multiple points : Done
* tmask : Done (or at least fake one done)
* off depth and onto grid (done)
* e1t etc (sketched)
* the surface (badly done)

In [ ]:
def interpolator(t_mask, e3w, e2v, e1u, w_coords, v_coords, u_coords, w, v, u, point) :
    dims = len(point)
    rhs = np.zeros((3))
    if t_mask[int(point[1]), int(point[2]), int(point[3])] != 0:
        for vel, scale, coords, data in zip([0, 1, 2], [e3w, e2v, e1u,], [w_coords, v_coords, u_coords], [w, v, u]):
            indices = []
            sub_coords = []
            for j in range(dims) :
                idx = np.digitize([point[j]], coords[j])[0]   # finds the index of region
                if (idx == len(coords[j])):
                    print (j, 'out of bounds', point[j], vel, coords[j])
                if (idx == 0):
                    print (j, 'hit surface', point[j], vel, coords[j])
                indices += [[idx - 1, idx]]
                sub_coords += [coords[j][indices[-1]]]
            indices = np.array([j for j in product(*indices)])
            sub_coords = np.array([j for j in product(*sub_coords)])
            sub_data = data[list(np.swapaxes(indices, 0, 1))]
            li = LinearNDInterpolator(sub_coords, sub_data)
            rhs[vel] = li([point])[0]/scale
    return rhs
# from Jaime on Stackoverflow https://stackoverflow.com/users/110026/jaime

In [ ]:
def derivatives(t, poss, t_mask, e3w, e2v, e1u, w_coords, v_coords, u_coords, w, v, u):
    rhs = np.zeros_like(poss)
    for ip in range(int(poss.shape[0]/3)):
        point = np.array([t, poss[0+ip*3], poss[1+ip*3], poss[2+ip*3]])
        rhs[0+ip*3:3+ip*3] = interpolator(t_mask, e3w, e2v, e1u, w_coords, v_coords, u_coords, w, v, u, point)
    return np.array(rhs) # array or scalar, not a tuple

In [ ]:
# Choose new random points
def random_points (t_coords, deltat):
    t0 = tcorrs[0]
    yi = np.zeros((27, 3))

    tc = random.uniform(t0, t0+deltat)
    zc = random.uniform(0., 1.)
    yc = random.uniform(t_coords[2, 0], t_coords[2, -1])
    xc = random.uniform(t_coords[3, 0], t_coords[3, -1])

    print (tc, zc, yc, xc)

    count = 0
    for k in range(3):
        for j in range(3):
            for i in range(3):
                yi[count, 0] = zc + k
                yi[count, 1] = yc + j
                yi[count, 2] = xc + i
                count += 1
    return yi

In [ ]:
# Set up run: test points are not on the walls
def test_points(t_coords, u_coords, v_coords, w_coords, tmask, yi):
    testpoint = np.zeros((4))
    for point in range(yi.shape[0])
        testpoint[1:] = yi[point]
        testpoint[0] = t0
        print (testpoint)
        velocity = interpolator(t_mask, e3w, e2v, e1u, w_coords, v_coords, u_coords, w, v, u, testpoint)
        print (velocity)
        if np.all(velocity == [0, 0, 0]):
            t_mask[int(testpoint[1]), int(testpoint[2]), int(testpoint[3])] = 0
            print (int(testpoint[1]), int(testpoint[2]), int(testpoint[3]), 'zerod')
    return tmask

In [ ]:
def w_correction (w, total_depth, depth):
    '''w in nc files, is real, but we want w relative to grid
    that means w  at the surface is zero'''
    stretching = w[:, 0]/total_depth
    for i in range(6):
        w[:, i] =- stretching*depth[:, i]
    return w

In [ ]:
def get_initial_data():
    udataset = nc.Dataset('/Users/sallen/Downloads/ubcSSn3DuVelocity1hV16-10_9411_b69f_35e1.nc')
    tcorrs = udataset['time'][:3]
    deltat = tcorrs[1] - tcorrs[0]
    xcorrs = udataset['gridX'][:]
    ycorrs = udataset['gridY'][:]
    depthsize = udataset['depth'][:].shape[0]
    zcorrs = np.linspace(0, depthsize, depthsize+1)  # this makes it depth, not grid point

    longaxis = max(len(xcorrs), len(ycorrs), len(zcorrs), len(tcorrs))
    print (longaxis)
    t_coords = np.zeros((4, longaxis))
    t_coords[0, 0:len(tcorrs)] = tcorrs
    t_coords[0, len(tcorrs):] = max(tcorrs)
    t_coords[1, 0:len(zcorrs)] = zcorrs
    t_coords[1, len(zcorrs):] = max(zcorrs)
    t_coords[2, 0:len(ycorrs)] = ycorrs
    t_coords[2, len(ycorrs):] = max(ycorrs)
    t_coords[3, 0:len(xcorrs)] = xcorrs
    t_coords[3, len(xcorrs):] = max(xcorrs)
    print (t_coords.shape)
    # other grids
    u_coords = np.copy(t_coords)
    u_coords[3] = t_coords[3] + 0.5
    v_coords = np.copy(t_coords)
    v_coords[2] = t_coords[2] + 0.5
    w_coords = np.copy(t_coords)
    w_coords[1] = t_coords[1] - 0.5

    u = np.zeros((3, len(zcorrs), len(ycorrs), len(xcorrs)))
    print (u.shape)
    u[:, 1:] = udataset['uVelocity'][0:3]
    u[:, 0] = 2 * u[:, 2] - u[:, 1]
    print (u.shape)

    v = np.zeros_like(u)
    vdataset = nc.Dataset('/Users/sallen/Downloads/ubcSSn3DvVelocity1hV16-10_a2b6_f6ad_26bd.nc')
    v[:, 1:] = vdataset['vVelocity'][0:3]
    v[:, 0] = 2 * v[:, 2] - v[:, 1]
    print (v.shape)

    wdataset = nc.Dataset('/Users/sallen/Downloads/ubcSSn3DwVelocity1hV16-10_9d37_aa6f_428c.nc')
    w = np.zeros_like(u)
    w = wdataset['wVelocity'][0:3]
    print(w.max(), w.min())
    #wdepths = g_depw
    wdepths = np.zeros((3, 6, 21, 21))
    for j in range(3):
        for i in range(6):
            wdepths[j, i] = wdataset['depth'][i]
    w = w_correction(w, 150., wdepths)
    print(w.shape)
    print(w.max(), w.min())

    nextindex = 3

    # t_mask (here because it will be read in the real version)
    t_mask = np.ones((40, 898, 398))
    print (t_mask.shape)
    t_mask[5:] = 0
    t_mask[:, 0:int(t_coords[2, 0]+1)] = 0
    t_mask[:, int(t_coords[2, -1]):] = 0
    t_mask[..., 0:int(t_coords[3, 0]+1)] = 0
    t_mask[..., int(t_coords[3, -1]):] = 0

    # set scales, for now
    e1u = 440
    e2v = 500
    e3w = 1
    return tmask, e1u, e2v, e3w, u, v, w, t_coords, u_coords, v_coords, w_coords, deltat, nextindex

In [ ]:
def update_arrays(tcorrs, u_coords, v_coords, w_coords, u, v, w, deltat, nextindex):
    tcorrs  = tcorrs + deltat
    u_coords[0, 0:len(tcorrs)] = tcorrs
    u_coords[0, len(tcorrs):] = max(tcorrs)
    v_coords[0] = u_coords[0]
    w_coords[0] = u_coords[0]
    u[0:2] = u[1:3]
    u[2, 1:] = udataset['uVelocity'][nextindex]
    u[2, 0] = 2 * u[2, 2] - u[2, 1]
    v[0:2] = v[1:3]
    v[2, 1:] = vdataset['vVelocity'][nextindex]
    v[2, 0] = 2 * v[2, 2] - v[2, 1]
    w[0:2] = w[1:3]
    w[2] = wdataset['wVelocity'][nextindex]
    nextindex += 1
    return tcorrs, u_coords, v_coords, w_coords, u, v, w, nextindex

In [ ]:
def myplotter(t0, y0, npoints, initial=False):
    if initial:
        for ip in range(npoints):
            ax[0, 1].plot(t0, y0[ip, 0], 'yo')
            ax[0, 0].plot(t0, y0[ip, 1], 'ys')
            ax[0, 0].plot(t0, y0[ip, 2], 'y^')
            ax[1, 0].plot(y0[ip, 2], y0[ip, 1], 'yo')
            ax[1, 1].scatter(y0[ip, 2], y0[ip, 1], y0[ip, 0], c='y')
    else:
        for ip in range(npoints):
            ax[0, 1].plot(t0, y0[ip, 0], 'bo')
            ax[0, 0].plot(t0, y0[ip, 1], 'bs')
            ax[0, 0].plot(t0, y0[ip, 2], 'b^')
            ax[1, 0].plot(y0[ip, 2], y0[ip, 1], 'bo')
            ax[1, 1].scatter(y0[ip, 2], y0[ip, 1], y0[ip, 0], c='by')

In [ ]:
# Main Cell

# set up graphing
fig, ax = plt.subplots(2, 2)    # initialize ax array
fig = plt.figure(figsize=(15, 15))
ax[0, 0] = fig.add_subplot(2, 2, 1)
ax[0, 1] = fig.add_subplot(2, 2, 2)
ax[1, 0] = fig.add_subplot(2, 2, 3)
ax[1, 1] = fig.add_subplot(2, 2, 4, projection='3d')

# get initial data
tmask, e1u, e2v, e3w, u, v, w, t_coords, u_coords, v_coords, w_coords, deltat, nextindex = get_initial_data()
# get a new point
yi, npoints = random_points (t_coords, deltat)
# test it (this version only) to build up mask
tmask = test_points(t_coords, u_coords, v_coords, w_coords, tmask, yi)

# set up values for integrator
t0 = tcorrs[0]
dt = deltat/10.
npoints = yi.shape[0]
y0 = np.copy(yi)
yp = np.ndarray.flatten(y0)

# plot initial points
myplotter(t0, y0, npoints, initial=True)

# initialize integrator    
myintegrator = ode(derivatives).set_integrator('dopri5', atol=0.01)
myintegrator.set_initial_value(yp, t0).set_f_params(t_mask, e3w, e2v, e1u, w_coords, v_coords, u_coords, w, v, u)

# first segment
t1 = t0 + 1.5*deltat

while myintegrator.successful() and myintegrator.t < t1:
    myintegrator.integrate(myintegrator.t + dt)
    myplotter(myintegrator.t, np.reshape(myintegrator.y, (npoints, 3)))

# and the rest
for count in range(8):
    # update arrays
    tcorrs, u_coords, v_coords, w_coords, u, v, w, nextindex = (
        update_arrays(tcorrs, u_coords, v_coords, w_coords, u, v, w, deltat, nextindex))
    t1 += deltat
  
    while myintegrator.successful() and myintegrator.t < t1:
        myintegrator.integrate(myintegrator.t + dt)
        myplotter(myintegrator.t, np.reshape(myintegrator.y, (npoints, 3)))

In [ ]:
udataset['depth'][:]

In [ ]:
zcorrs = np.linspace(0, depthsize, depthsize+1)

In [ ]:
wdataset